In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import ast
import re

import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# nltk.download('all')

# from src.utils.helpers_news_movies import *
# from src.utils.movies_genres_utils import *

In [2]:
df_plot = pd.read_csv('data/MovieSummaries/movies_metadata_cleaned.csv')
df_plot = df_plot[df_plot['Decade'].notna()]
df_plot = df_plot[df_plot['Grouped_genres'] != '[nan]']
df_10 = df_plot.head(10)
df_10

,Unnamed: 0.1,Unnamed: 0,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Decade,Movie_continent,Grouped_genres,Plot_summary
0,0,0,975900,/m/03vyhn,Ghosts of Mars,2001.0,14010832.0,98.0,['English Language'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",2000.0,North America,"['Science Fiction', 'Supernatural', 'Thriller'...",Set second half 22nd century film depicts Mars...
1,1,3,9363483,/m/0285_cd,White Of The Eye,1987.0,NaN,110.0,['English Language'],['United Kingdom'],"['Thriller', 'Erotic thriller', 'Psychological...",1980.0,Europe,['Thriller'],series murders rich young women throughout Ari...
2,2,4,261236,/m/01mrr1,A Woman in Flames,1983.0,NaN,106.0,['German Language'],['Germany'],['Drama'],1980.0,Europe,['Drama'],Eva upper class housewife becomes frustrated l...
3,3,6,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002.0,NaN,86.0,['English Language'],['South Africa'],"['Family Film', 'Fantasy', 'Adventure', 'World...",2000.0,Africa,"['Fantasy', 'Family', 'Action/Adventure', 'Wor...",Every hundred years evil Morgana returns claim...
4,4,12,6631279,/m/0gffwj,Little city,1997.0,NaN,93.0,['English Language'],['United States of America'],"['Romantic comedy', 'Ensemble Film', 'Comedy-d...",1990.0,North America,"['Ensemble Film', 'Romance', 'Comedy', 'Drama']",Adam San Franciscobased artist works cab drive...
5,5,13,171005,/m/016ywb,Henry V,1989.0,10161099.0,137.0,['English Language'],['United Kingdom'],"['Costume drama', 'War film', 'Epic', 'Period ...",1980.0,Europe,"['Drama', 'Epic', 'Period', 'War', 'Film adapt...",PlotdateAct 1Act 2Act 3Act 4Act 5 Finally nego...
6,6,14,18296435,/m/04cqrs4,Aaah Belinda,1986.0,NaN,NaN,['Turkish Language'],['Turkey'],['Comedy'],1980.0,Asia,['Comedy'],Serap young actress strong lively personality ...
8,8,17,77856,/m/0kcn7,Mary Poppins,1964.0,102272727.0,139.0,['English Language'],['United States of America'],"[""Children's/Family"", 'Musical', 'Fantasy', 'C...",1960.0,North America,"['Comedy', 'Drama', 'Children', 'Fantasy', 'Mu...",film opens Mary Poppins perched cloud high Lon...
10,10,22,21926710,/m/05p45cv,White on Rice,2009.0,NaN,82.0,[],['United States of America'],"['Romantic comedy', 'Romance Film', 'Comedy', ...",2000.0,North America,"['Indie', 'Romance', 'Comedy']",Jimmy Hiroshi Watanabe loves dinosaurs sleeps ...
11,11,27,20604092,/m/0523t_1,Anbu Thozhi,2007.0,NaN,NaN,['Tamil Language'],['India'],['Romance Film'],2000.0,Asia,['Romance'],film opens simpleton small village Tamil Nadu ...


In [3]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)

    return processed_text

df_10['Plot_summary'] = df_10['Plot_summary'].apply(preprocess_text)
df_10

/var/folders/_q/jt75w9f50x39z9gk_wgh16l80000gn/T/ipykernel_14502/3951617887.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['Plot_summary'] = df_10['Plot_summary'].apply(preprocess_text)


,Unnamed: 0.1,Unnamed: 0,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Decade,Movie_continent,Grouped_genres,Plot_summary
0,0,0,975900,/m/03vyhn,Ghosts of Mars,2001.0,14010832.0,98.0,['English Language'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",2000.0,North America,"['Science Fiction', 'Supernatural', 'Thriller'...",set second half 22nd century film depicts mar ...
1,1,3,9363483,/m/0285_cd,White Of The Eye,1987.0,NaN,110.0,['English Language'],['United Kingdom'],"['Thriller', 'Erotic thriller', 'Psychological...",1980.0,Europe,['Thriller'],series murder rich young woman throughout ariz...
2,2,4,261236,/m/01mrr1,A Woman in Flames,1983.0,NaN,106.0,['German Language'],['Germany'],['Drama'],1980.0,Europe,['Drama'],eva upper class housewife becomes frustrated l...
3,3,6,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002.0,NaN,86.0,['English Language'],['South Africa'],"['Family Film', 'Fantasy', 'Adventure', 'World...",2000.0,Africa,"['Fantasy', 'Family', 'Action/Adventure', 'Wor...",every hundred year evil morgana return claim f...
4,4,12,6631279,/m/0gffwj,Little city,1997.0,NaN,93.0,['English Language'],['United States of America'],"['Romantic comedy', 'Ensemble Film', 'Comedy-d...",1990.0,North America,"['Ensemble Film', 'Romance', 'Comedy', 'Drama']",adam san franciscobased artist work cab driver...
5,5,13,171005,/m/016ywb,Henry V,1989.0,10161099.0,137.0,['English Language'],['United Kingdom'],"['Costume drama', 'War film', 'Epic', 'Period ...",1980.0,Europe,"['Drama', 'Epic', 'Period', 'War', 'Film adapt...",plotdateact 1act 2act 3act 4act 5 finally nego...
6,6,14,18296435,/m/04cqrs4,Aaah Belinda,1986.0,NaN,NaN,['Turkish Language'],['Turkey'],['Comedy'],1980.0,Asia,['Comedy'],serap young actress strong lively personality ...
8,8,17,77856,/m/0kcn7,Mary Poppins,1964.0,102272727.0,139.0,['English Language'],['United States of America'],"[""Children's/Family"", 'Musical', 'Fantasy', 'C...",1960.0,North America,"['Comedy', 'Drama', 'Children', 'Fantasy', 'Mu...",film open mary poppins perched cloud high lond...
10,10,22,21926710,/m/05p45cv,White on Rice,2009.0,NaN,82.0,[],['United States of America'],"['Romantic comedy', 'Romance Film', 'Comedy', ...",2000.0,North America,"['Indie', 'Romance', 'Comedy']",jimmy hiroshi watanabe love dinosaur sleep top...
11,11,27,20604092,/m/0523t_1,Anbu Thozhi,2007.0,NaN,NaN,['Tamil Language'],['India'],['Romance Film'],2000.0,Asia,['Romance'],film open simpleton small village tamil nadu w...


In [4]:
# Initialize NLTK sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Create get_sentiment function
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0
    return sentiment

# Apply get_sentiment function
df_10['NLTK_Sentiment'] = df_10['Plot_summary'].apply(get_sentiment)
df_10

/var/folders/_q/jt75w9f50x39z9gk_wgh16l80000gn/T/ipykernel_14502/1944751056.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['NLTK_Sentiment'] = df_10['Plot_summary'].apply(get_sentiment)


,Unnamed: 0.1,Unnamed: 0,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Decade,Movie_continent,Grouped_genres,Plot_summary,NLTK_Sentiment
0,0,0,975900,/m/03vyhn,Ghosts of Mars,2001.0,14010832.0,98.0,['English Language'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...",2000.0,North America,"['Science Fiction', 'Supernatural', 'Thriller'...",set second half 22nd century film depicts mar ...,1
1,1,3,9363483,/m/0285_cd,White Of The Eye,1987.0,NaN,110.0,['English Language'],['United Kingdom'],"['Thriller', 'Erotic thriller', 'Psychological...",1980.0,Europe,['Thriller'],series murder rich young woman throughout ariz...,1
2,2,4,261236,/m/01mrr1,A Woman in Flames,1983.0,NaN,106.0,['German Language'],['Germany'],['Drama'],1980.0,Europe,['Drama'],eva upper class housewife becomes frustrated l...,1
3,3,6,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002.0,NaN,86.0,['English Language'],['South Africa'],"['Family Film', 'Fantasy', 'Adventure', 'World...",2000.0,Africa,"['Fantasy', 'Family', 'Action/Adventure', 'Wor...",every hundred year evil morgana return claim f...,1
4,4,12,6631279,/m/0gffwj,Little city,1997.0,NaN,93.0,['English Language'],['United States of America'],"['Romantic comedy', 'Ensemble Film', 'Comedy-d...",1990.0,North America,"['Ensemble Film', 'Romance', 'Comedy', 'Drama']",adam san franciscobased artist work cab driver...,1
5,5,13,171005,/m/016ywb,Henry V,1989.0,10161099.0,137.0,['English Language'],['United Kingdom'],"['Costume drama', 'War film', 'Epic', 'Period ...",1980.0,Europe,"['Drama', 'Epic', 'Period', 'War', 'Film adapt...",plotdateact 1act 2act 3act 4act 5 finally nego...,1
6,6,14,18296435,/m/04cqrs4,Aaah Belinda,1986.0,NaN,NaN,['Turkish Language'],['Turkey'],['Comedy'],1980.0,Asia,['Comedy'],serap young actress strong lively personality ...,1
8,8,17,77856,/m/0kcn7,Mary Poppins,1964.0,102272727.0,139.0,['English Language'],['United States of America'],"[""Children's/Family"", 'Musical', 'Fantasy', 'C...",1960.0,North America,"['Comedy', 'Drama', 'Children', 'Fantasy', 'Mu...",film open mary poppins perched cloud high lond...,1
10,10,22,21926710,/m/05p45cv,White on Rice,2009.0,NaN,82.0,[],['United States of America'],"['Romantic comedy', 'Romance Film', 'Comedy', ...",2000.0,North America,"['Indie', 'Romance', 'Comedy']",jimmy hiroshi watanabe love dinosaur sleep top...,1
11,11,27,20604092,/m/0523t_1,Anbu Thozhi,2007.0,NaN,NaN,['Tamil Language'],['India'],['Romance Film'],2000.0,Asia,['Romance'],film open simpleton small village tamil nadu w...,1


In [5]:
from textblob import TextBlob

# Perform sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # Returns a polarity score (-1 to 1)

df_10['TB_Sentiment'] = df_10['Plot_summary'].apply(get_sentiment)
df_10[['TB_Sentiment', 'Plot_summary']]

/var/folders/_q/jt75w9f50x39z9gk_wgh16l80000gn/T/ipykernel_14502/1842718526.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['TB_Sentiment'] = df_10['Plot_summary'].apply(get_sentiment)


,TB_Sentiment,Plot_summary
0,-0.075099,set second half 22nd century film depicts mar ...
1,0.039072,series murder rich young woman throughout ariz...
2,0.152296,eva upper class housewife becomes frustrated l...
3,-0.036790,every hundred year evil morgana return claim f...
4,0.162716,adam san franciscobased artist work cab driver...
5,-0.007143,plotdateact 1act 2act 3act 4act 5 finally nego...
6,0.006017,serap young actress strong lively personality ...
8,0.122749,film open mary poppins perched cloud high lond...
10,0.423003,jimmy hiroshi watanabe love dinosaur sleep top...
11,0.270588,film open simpleton small village tamil nadu w...


In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import download

download('vader_lexicon')  # Ensure VADER is downloaded
analyzer = SentimentIntensityAnalyzer()

# Perform sentiment analysis
df_10['VADER_Sentiment'] = df_10['Plot_summary'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
df_10[['Plot_summary', 'VADER_Sentiment', 'TB_Sentiment']]

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/shulijia/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/var/folders/_q/jt75w9f50x39z9gk_wgh16l80000gn/T/ipykernel_14502/2402501819.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['VADER_Sentiment'] = df_10['Plot_summary'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


,Plot_summary,VADER_Sentiment,TB_Sentiment
0,set second half 22nd century film depicts mar ...,-0.9880,-0.075099
1,series murder rich young woman throughout ariz...,-0.9975,0.039072
2,eva upper class housewife becomes frustrated l...,0.9716,0.152296
3,every hundred year evil morgana return claim f...,-0.8807,-0.036790
4,adam san franciscobased artist work cab driver...,0.7351,0.162716
5,plotdateact 1act 2act 3act 4act 5 finally nego...,0.3182,-0.007143
6,serap young actress strong lively personality ...,-0.6908,0.006017
8,film open mary poppins perched cloud high lond...,0.9910,0.122749
10,jimmy hiroshi watanabe love dinosaur sleep top...,0.9643,0.423003
11,film open simpleton small village tamil nadu w...,0.9861,0.270588


In [7]:
from nrclex import NRCLex

# Ensure the column is converted to string type to handle potential NaNs
df_10['Plot_summary'] = df_10['Plot_summary'].astype(str)

# Function to get dominant emotions
def extract_emotions(text):
    try:
        emotion = NRCLex(text)
        return emotion.raw_emotion_scores  
    except Exception as e:
        return str(e)  # Return error message if any issues occur

# Apply the emotion extraction to the 'Plot_summary' column
df_10['Dominant_Emotions'] = df_10['Plot_summary'].apply(extract_emotions)

# Display the DataFrame with the extracted emotions
df_10[['Plot_summary', 'Dominant_Emotions']].to_csv('emotions_output.csv', index=False)

/var/folders/_q/jt75w9f50x39z9gk_wgh16l80000gn/T/ipykernel_14502/1596710759.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['Plot_summary'] = df_10['Plot_summary'].astype(str)
/var/folders/_q/jt75w9f50x39z9gk_wgh16l80000gn/T/ipykernel_14502/1596710759.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['Dominant_Emotions'] = df_10['Plot_summary'].apply(extract_emotions)


In [ ]:
# from transformers import pipeline

# # Load pre-trained sentiment analysis pipeline with explicit model specification
# sentiment_analyzer = pipeline(
#     "sentiment-analysis",
#     model="distilbert-base-uncased-finetuned-sst-2-english",
#     truncation=True
# )

# # Apply sentiment analysis on the 'Plot_summary' column of your dataframe
# # Assuming df_plot['Plot_summary'] is a Pandas Series containing text
# sentiments = df_10['Plot_summary'].apply(lambda text: sentiment_analyzer(text)[0])

# # Add sentiment results back to the dataframe
# df_10['HFT_Sentiment'] = sentiments.apply(lambda x: x['label'])
# df_10['HFT_Score'] = sentiments.apply(lambda x: x['score'])

# # Display the updated dataframe
# df_10[['Plot_summary', 'HFT_Sentiment', 'HFT_Score', 'TB_Sentiment', 'VADER_Sentiment']]

from transformers import pipeline

# Load pre-trained sentiment analysis pipeline
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    truncation=True
)

# Function to analyze sentiments in batches
def analyze_sentiments_in_batches(texts, batch_size=16):
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        results.extend(sentiment_analyzer(batch))
    return results

# Apply sentiment analysis on the 'Plot_summary' column
df_10['HFT_Sentiment_Result'] = analyze_sentiments_in_batches(df_10['Plot_summary'].tolist())
df_10['HFT_Sentiment'] = df_10['HFT_Sentiment_Result'].apply(lambda x: x['label'])
df_10['HFT_Score'] = df_10['HFT_Sentiment_Result'].apply(lambda x: x['score'])

# Display the updated dataframe
print(df_10[['Plot_summary', 'HFT_Sentiment_Result', 'HFT_Sentiment', 'HFT_Score']])

/opt/miniconda3/envs/ADA/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/miniconda3/envs/ADA/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/miniconda3/envs/ADA/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
